In [1]:
import pandas as pd
import numpy as np
import os

In [17]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [158]:
ALL_EVALUATIONS="tests/evaluation_04_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20231211.tsv"
BM25="../trabalho_final/validation_queries_retrievals/200-queries-passagens-by-BM25-pt_v2.tsv"
BM25_E5="tests/bm25_e5_docs_000_999_50_sampled_queries.tsv"
BM25_MT5="../trabalho_final/validation_queries_retrievals/200-queries-passagens-by-BM25-mT5_v4_with_query_id.tsv"
SPLADE="../trabalho_final/validation_queries_retrievals/splade_pt_result_with_clueweb_id.tsv"
COLBERTX="../trabalho_final/validation_queries_retrievals/colbertx_all_znormalized_validation_scores_000_199.tsv"

In [218]:
retrievers_results = []

### Format GPT-4 evaluations to qrels format

In [3]:
eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

In [4]:
eval_df

,query_id,query,passage_id,mT5,splade,colbertx,bm25_e5,passage,score,reasoning,usage,cost,duration,saved_cost
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-82-13187_0,False,False,True,False,Mamíferos do Brasil - Zoologia | Museu Naciona...,1,A passagem lista vários mamíferos da fauna bra...,"{'prompt_tokens': 765, 'completion_tokens': 75...",0.00000,6.693229,0.02745
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-59-04599_2,False,False,True,False,Segundo uma lista levantada pelo Comitê Brasil...,1,A passagem descreve várias espécies de aves en...,"{'prompt_tokens': 860, 'completion_tokens': 82...",0.00000,5.163547,0.03072
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_39,True,True,False,False,"A exuberante fauna e flora brasileiras, relata...",3,A passagem menciona que o Brasil tem a maior b...,"{'prompt_tokens': 840, 'completion_tokens': 46...",0.00000,5.185688,0.02796
3,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-77-06884_1,False,True,True,False,O tamanho do Brasil e a grande variedade de cl...,1,A passagem menciona a alta biodiversidade do B...,"{'prompt_tokens': 812, 'completion_tokens': 81...",0.00000,5.882017,0.02922
4,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-78-09747_0,True,False,False,False,Onça-pintada - Escola Kids Qual matéria está p...,1,A passagem menciona que a onça-pintada é um do...,"{'prompt_tokens': 805, 'completion_tokens': 96...",0.00000,5.774773,0.02991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0001-84-17796_0,True,True,True,True,Artistas Impressionistas: lista dos principais...,3,A passagem fornece uma lista detalhada dos pri...,"{'prompt_tokens': 888, 'completion_tokens': 41...",0.00000,2.660084,0.02910
1285,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0001-84-17796_1,False,False,True,False,- Francesco Filippini (pintor italiano) - Gius...,2,"A passagem lista vários artistas, incluindo al...","{'prompt_tokens': 820, 'completion_tokens': 87...",0.00000,7.354066,0.02982
1286,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0001-70-17886_9,False,False,False,True,São duas as fontes mais importantes do impress...,3,"A passagem responde diretamente à pergunta, li...","{'prompt_tokens': 808, 'completion_tokens': 48...",0.02712,4.073626,0.00000
1287,199,Quais foram os principais artistas do Impressi...,clueweb22-pt0000-79-11166_1,False,False,False,True,O impressionismo foi um movimento de arte que ...,3,"A passagem responde diretamente à pergunta, li...","{'prompt_tokens': 741, 'completion_tokens': 67...",0.02625,7.706426,0.00000


In [132]:
eval_df.drop_duplicates("query_id")[['query_id', 'query']]

,query_id,query
0,1,Qual a maior característica da fauna brasileira?
29,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
57,9,quais países europeus seguem o regime monarquista?
85,11,quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?
109,13,Como o Brasil reagiu a epidemia de AIDS no fim do século XX?
134,15,Qual a maior torcida de futebol do Brasil?
151,17,Quando se realizou o plebiscito popular para definir o sistema político do Brasil?
171,20,Por que a legislação de um país é tão importante?
204,21,Qual o nosso atual sistema de governo?
237,22,Qual o sistema de governo da África do Sul?


#### Check scores distribution on the current qrels

In [103]:
np.unique(eval_df['score'].to_numpy(), return_counts=True)

(array([0, 1, 2, 3]), array([435, 247, 252, 355]))

In [104]:
for group_name, group_df in eval_df.groupby('query_id'):
    print("\nquery_id={}".format(group_name))
    
    print(np.unique(group_df['score'].to_numpy(), return_counts=True))


query_id=1
(array([0, 1, 2, 3]), array([ 5, 11,  8,  5]))

query_id=2
(array([0, 1, 2, 3]), array([23,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([15,  3,  8,  2]))

query_id=11
(array([0, 1, 2, 3]), array([10,  6,  6,  2]))

query_id=13
(array([0, 1, 2, 3]), array([6, 8, 7, 4]))

query_id=15
(array([1, 3]), array([ 7, 10]))

query_id=17
(array([0, 2, 3]), array([ 2,  1, 17]))

query_id=20
(array([0, 1, 2, 3]), array([ 3, 14, 13,  3]))

query_id=21
(array([0, 1, 2, 3]), array([12,  8,  1, 12]))

query_id=22
(array([0, 1, 3]), array([15,  4,  4]))

query_id=24
(array([0, 1, 2, 3]), array([ 6, 14,  8,  2]))

query_id=26
(array([1, 2, 3]), array([ 1,  5, 18]))

query_id=28
(array([0, 1, 2, 3]), array([11,  5,  7,  4]))

query_id=47
(array([0, 1, 2]), array([10, 11,  9]))

query_id=49
(array([0, 1, 2, 3]), array([10,  7, 10,  1]))

query_id=51
(array([0, 1, 2, 3]), array([ 6, 10,  3,  1]))

query_id=54
(array([1, 2, 3]), array([ 3,  4, 22]))

query_id=60
(array([0, 1, 3]), arra

In [92]:
eval_df.groupby('query_id').count()

,query,passage_id,mT5,splade,colbertx,bm25_e5,passage,score,reasoning,usage,cost,duration,saved_cost,0
query_id,,,,,,,,,,,,,,
1,29,29,29,29,29,29,29,29,29,29,29,29,29,29
2,28,28,28,28,28,28,28,28,28,28,28,28,28,28
9,28,28,28,28,28,28,28,28,28,28,28,28,28,28
11,24,24,24,24,24,24,24,24,24,24,24,24,24,24
13,25,25,25,25,25,25,25,25,25,25,25,25,25,25
15,17,17,17,17,17,17,17,17,17,17,17,17,17,17
17,20,20,20,20,20,20,20,20,20,20,20,20,20,20
20,33,33,33,33,33,33,33,33,33,33,33,33,33,33
21,33,33,33,33,33,33,33,33,33,33,33,33,33,33


#### Prepare the qrels format for trec_eval tools

In [100]:
eval_df['0'] = '0'

In [93]:
passages_order = list(range(0, eval_df.shape[0]))

In [95]:
np.random.shuffle(passages_order)

In [101]:
eval_df.iloc[passages_order].sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv("tests/evaluation_04_retrievers_00_009_sampled_50_queries_gpt-4-1106-preview_20231211_shuffled_qrel.txt", header=None, index=False, sep=" ")

### Prepare BM25 results in run format

In [159]:
bm25_df = pd.read_csv(BM25, sep="\t")

In [162]:
bm25_df.groupby('query').count()

,passagem-id,passagem-by-BM25
query,,
As primaveiras estão sendo antessipadas?,10,10
Brasileiros conseguem trabalhar em Portugal?,10,10
Carne de capivara é saborosa?,10,10
Como José Sarney se tornou presidente do Brasil?,10,10
Como a cultura por influenciar o pensamento de uma geração?,10,10
Como a técnica de normalização ajudam pacientes psicóticos?,10,10
Como baixar arquivos do OneDrive?,10,10
Como configurar uma linguagem do teclado windowns 10?,10,10
Como definir a missão de uma empresa?,10,10


In [164]:
queries_01 = pd.read_csv("../trabalho_final/validation_queries_retrievals/queries_validation.tsv", sep='\t', header=None, names=['query_id', 'query'])
queries_02 = pd.read_csv("../trabalho_final/validation_queries_retrievals/validation_queries_mirelle.tsv", sep='\t', header=None, names=['query_id', 'query'])

In [165]:
queries_01.head()

,query_id,query
0,0,Qual a maior característica da flora brasileira?
1,1,Qual a maior característica da fauna brasileira?
2,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
3,3,Por que a Bolívia tem duas capitais?
4,4,Qual o que significa decolonialidade?


In [166]:
queries_02.head()

,query_id,query
0,100,Qual objetivo da Enel Brasil?
1,101,Qual a abordagem de mineração da moeda digital Chia (XCH)?
2,102,Quais produtos são revendidos pela empresa Casa da Coifa?
3,103,Em que ano surgiu a Universities and Colleges Admissions Service (UCAS)?
4,104,Quais etapas para a inscrição pelo Colleges Admissions Service (UCAS)?


In [170]:
all_queries_df = pd.concat([queries_01, queries_02])

In [171]:
all_queries_df.shape

(200, 2)

In [174]:
bm25_df = bm25_df.merge(all_queries_df)

In [175]:
bm25_df

,query,passagem-id,passagem-by-BM25,query_id
0,Qual objetivo da Enel Brasil?,clueweb22-pt0001-44-18816_1,"Nos últimos anos, o atendimento ao cliente Enel, tem se aprimorado, com o objetivo de facilitar ainda mais a vida dos consumidores. Com isso, grande parte dos serviços de atendimento ao cliente Enel, podem ser realizados de maneira prática, por meio da internet. Através do aplicativo Enel- Que está disponível para download em aparelhos com Android ou IO Nas Redes Sociais – Como na página do Facebook (Enel clientes Brasil). Ou no Twitter (@ Enel clientes BR). Por meio do site- Na página da Enel, (www.enel.com.br/pt.html). O atendimento Enel ao cliente, pode ser feito por meio de um assistente virtual. Veja o vídeo: Outras formas de atendimento Além de poder receber o atendimento por meio dos canais online, você também pode obter atendimento nos Estados onde a Enel Brasil está presente. Por meio do telefone de atendimento Enel 0800, ou indo pessoalmente a uma loja de atendimento mais próxima da sua casa. Número 0800 para atendimento da Enel Pelo telefone de atendimento ao cliente da Enel, é possível esclarecer suas dúvidas e ainda relatar algum problema, entre outros serviços. Veja quais são os telefones da Enel atendimento ao cliente, aproveite para anotar o número da Enel em seu estado: Atendimento ao cliente Enel Ceará: 0800 285 0196",100
1,Qual objetivo da Enel Brasil?,clueweb22-pt0001-44-10630_6,"A ouvidoria é um canal de segunda instância que tem o objetivo de ouvir e tentar resolver problemas/solicitações que os canais convencionais de atendimento não foram capazes de solucionar. Caso nem mesmo a ouvidoria da Enel seja capaz de resolver em até 15 dias sua manifestação, é indicado que você busque ajuda dos órgãos responsáveis pela defesa do consumidor. 8. Redes sociais da Enel Brasil Outra forma de buscar falar com Enel é através de algumas redes sociais. A empresa geralmente auxilia os clientes pelo Facebook e Twitter, seja em comentários públicos ou mensagens privadas. Além disso, elas são boas alternativas para que você acompanhe as principais novidades e mudanças da distribuidora de energia. Os links da redes sociais da Enel Brasil são: Facebook Twitter Instagram (não presta suporte) 9. Resumo dos contatos da Enel Veja na tabela a seguir um resumo com alguns dos principais contatos oficiais da Enel abordados no artigo. Além disso, também listamos contatos que a empresa não disponibiliza para que você não caia em golpes. Se ficar dúvidas sobre algum dos meios de atendimento abaixo, leia os tópicos do artigo novamente e verá informações mais detalhadas. Fale com a Enel Contato SÃO PAULO Telefone Enel São Paulo (SAC) 0800-727-2120 Telefone para ligações de emergência",100
2,Qual objetivo da Enel Brasil?,clueweb22-pt0001-05-18926_3,"O medidor inteligente, aprovado pelo Instituto Nacional de Metrologia, Qualidade e Tecnologia (INMETRO), foi desenvolvido para atender às necessidades locais, respeitando as características da rede elétrica e os requisitos das normas técnicas. Globalmente, a Enel já instalou mais de 45 milhões de medidores inteligentes em países como Argentina, Chile, Colômbia, Itália, Peru, Romênia e Espanha. Sobre a Enel Distribuição São Paulo A Enel Distribuição São Paulo é uma empresa da multinacional de energia Enel. A companhia é a segunda maior distribuidora do país, respondendo por 10,3% de toda energia distribuída no Brasil e atendendo 7,4 milhões de unidades consumidoras em 24 municípios da região metropolitana de São Paulo, incluindo a capital paulista, principal centro econômico-financeiro do Brasil. A estratégia de atuação da Enel é baseada no seu Plano de Sustentabilidade e nos compromissos assumidos com os Objetivos de Desenvolvimento Sustentável da Agenda 2030 da ONU. |8 de Setembro de 2021 Enel São Paulo instalou 89 mil medidores inteligentes em bairros da Zona Oeste Pdf (0.1MB) DOWNLOAD [EDIT TITLE]",100
3,Qual objetivo da Enel Brasil?,clueweb22-pt0000-62-20856_1,"Serviço: Loja de Atendime

In [176]:
bm25_df.to_csv("../trabalho_final/validation_queries_retrievals/200-queries-passagens-by-BM25-pt_v2_with_query_id.tsv")

In [177]:
bm25_df['Q0'] = "Q0"
bm25_df['comment'] = "BM25"

In [178]:
bm25_top_10_df = bm25_df.groupby('query_id').head(10).reset_index(drop=True)

In [179]:
bm25_top_10_df.shape

(1990, 6)

In [181]:
bm25_top_10_df['position'] = np.tile(list(range(1, 11)), 199)
bm25_top_10_df['score'] = np.tile(list(range(100, 0, -10)), 199)

In [182]:
bm25_top_10_df.head(12)

,query,passagem-id,passagem-by-BM25,query_id,Q0,comment,position,score
0,Qual objetivo da Enel Brasil?,clueweb22-pt0001-44-18816_1,"Nos últimos anos, o atendimento ao cliente Enel, tem se aprimorado, com o objetivo de facilitar ainda mais a vida dos consumidores. Com isso, grande parte dos serviços de atendimento ao cliente Enel, podem ser realizados de maneira prática, por meio da internet. Através do aplicativo Enel- Que está disponível para download em aparelhos com Android ou IO Nas Redes Sociais – Como na página do Facebook (Enel clientes Brasil). Ou no Twitter (@ Enel clientes BR). Por meio do site- Na página da Enel, (www.enel.com.br/pt.html). O atendimento Enel ao cliente, pode ser feito por meio de um assistente virtual. Veja o vídeo: Outras formas de atendimento Além de poder receber o atendimento por meio dos canais online, você também pode obter atendimento nos Estados onde a Enel Brasil está presente. Por meio do telefone de atendimento Enel 0800, ou indo pessoalmente a uma loja de atendimento mais próxima da sua casa. Número 0800 para atendimento da Enel Pelo telefone de atendimento ao cliente da Enel, é possível esclarecer suas dúvidas e ainda relatar algum problema, entre outros serviços. Veja quais são os telefones da Enel atendimento ao cliente, aproveite para anotar o número da Enel em seu estado: Atendimento ao cliente Enel Ceará: 0800 285 0196",100,Q0,BM25,1,100
1,Qual objetivo da Enel Brasil?,clueweb22-pt0001-44-10630_6,"A ouvidoria é um canal de segunda instância que tem o objetivo de ouvir e tentar resolver problemas/solicitações que os canais convencionais de atendimento não foram capazes de solucionar. Caso nem mesmo a ouvidoria da Enel seja capaz de resolver em até 15 dias sua manifestação, é indicado que você busque ajuda dos órgãos responsáveis pela defesa do consumidor. 8. Redes sociais da Enel Brasil Outra forma de buscar falar com Enel é através de algumas redes sociais. A empresa geralmente auxilia os clientes pelo Facebook e Twitter, seja em comentários públicos ou mensagens privadas. Além disso, elas são boas alternativas para que você acompanhe as principais novidades e mudanças da distribuidora de energia. Os links da redes sociais da Enel Brasil são: Facebook Twitter Instagram (não presta suporte) 9. Resumo dos contatos da Enel Veja na tabela a seguir um resumo com alguns dos principais contatos oficiais da Enel abordados no artigo. Além disso, também listamos contatos que a empresa não disponibiliza para que você não caia em golpes. Se ficar dúvidas sobre algum dos meios de atendimento abaixo, leia os tópicos do artigo novamente e verá informações mais detalhadas. Fale com a Enel Contato SÃO PAULO Telefone Enel São Paulo (SAC) 0800-727-2120 Telefone para ligações de emergência",100,Q0,BM25,2,90
2,Qual objetivo da Enel Brasil?,clueweb22-pt0001-05-18926_3,"O medidor inteligente, aprovado pelo Instituto Nacional de Metrologia, Qualidade e Tecnologia (INMETRO), foi desenvolvido para atender às necessidades locais, respeitando as características da rede elétrica e os requisitos das normas técnicas. Globalmente, a Enel já instalou mais de 45 milhões de medidores inteligentes em países como Argentina, Chile, Colômbia, Itália, Peru, Romênia e Espanha. Sobre a Enel Distribuição São Paulo A Enel Distribuição São Paulo é uma empresa da multinacional de energia Enel. A companhia é a segunda maior distribuidora do país, respondendo por 10,3% de toda energia distribuída no Brasil e atendendo 7,4 milhões de unidades consumidoras em 24 municípios da região metropolitana de São Paulo, incluindo a capital paulista, principal centro econômico-financeiro do Brasil. A estratégia de atuação da Enel é baseada no seu Plano de Sustentabilidade e nos compromissos assumidos com os Objetivos de Desenvolvimento Sustentável da Agenda 2030 da ONU. |8 de Setembro de 2021 Enel São Paulo instalou 89 mil medidores inteligentes em bairros da Zona Oeste Pdf (0.1MB) DOWNLOAD [EDIT TITLE]",100,Q0,BM25,3,80
3,Qual objetivo da E

In [184]:
bm25_top_10_df[['query_id', 'Q0', "passagem-id", "position", "score", "comment"]].to_csv("tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt", header=None, index=False, sep=" ")

In [236]:
bm25_top_10_df.columns

Index(['query', 'passagem-id', 'passagem-by-BM25', 'query_id', 'Q0', 'comment',
       'position', 'score'],
      dtype='object')

In [237]:
bm25_top_10_df.columns = ['query', 'passage_id', 'passage', 'query_id', 'Q0', 'comment', 'position', 'score']

In [241]:
bm25_top_10_df[['query', 'passage_id', 'passage', 'query_id', 'score']].to_csv("tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_for_evaluation.tsv", index=False, sep='\t')

(1990, 8)

#### Compute NDCG@10

In [219]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    tests/evaluation_04_retrievers_00_009_sampled_50_queries_gpt-4-1106-preview_20231211_qrel.txt tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [220]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare BM25 + E5 retrieval results in run format

In [71]:
bm25_e5_df = pd.read_csv(BM25_E5, sep="\t")

In [72]:
bm25_e5_df.groupby("query_id").head(10).count()

query          500
passage_id     500
score          500
query_id       500
colbertx_id    500
passage        500
dtype: int64

In [73]:
bm25_e5_run_df = bm25_e5_df.sort_values(["query_id", "score"], ascending=[True, False]).groupby("query_id").head(10)

In [74]:
bm25_e5_run_df.shape

(500, 6)

In [75]:
bm25_e5_run_df.head()

,query,passage_id,score,query_id,colbertx_id,passage
45085,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_0,88.380936,1,5877599,"Fauna do Brasil - Toda Matéria Biologia › Zoologia Fauna do Brasil A fauna do Brasil é provavelmente a que possui a maior diversidade em espécies de animais vertebrados, anfíbios e primatas do globo terrestre. Existem três vezes mais espécies de peixes de água doce no Brasil do que em qualquer lugar no mundo. As aves também são muito variadas, colocando o Brasil como o país com a maior variedade de espécies na América do Sul e o terceiro do mundo em espécies de mamíferos. De igual modo, a Floresta Atlântica está entre as cinco regiões do planeta com mais espécies nativas, das quais 17% são aves e 10% são anfíbios e mamíferos encontrados somente na fauna brasileira. Vale citar que os animais da fauna brasileira são, via de regra, de pequeno porte. A importância científica da fauna brasileira é indescritível, pois contém a maior biodiversidade do mundo (cerca de 100 mil espécies de invertebrados, 8200 espécies vertebrados (713 mamíferos, 1826 aves, 721 répteis, 875 anfíbios, 2800 peixes continentais e 1300 peixes marinhos) das quais 627 já estão na lista negra das espécies ameaçadas de extinção (394 espécies terrestres e 233 espécies aquáticas)."
45086,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-96-00412_0,85.875961,1,4583617,"15 ANIMAIS BRASILEIROS: Características e curiosidades Animais brasileiros: nativos, endêmicos e em risco de extinção Fauna quer dizer o conjunto de espécies que vivem em uma determinada região. Por isso, é importante saber diferenciar que quando falamos sobre a fauna brasileira, estamos falando sobre todas as espécies que habitam o Brasil, mas não necessariamente apenas de espécies endêmicas ou nativas, já que alguns animais são considerados espécies invasoras e/ou foram introduzidos pela pessoa humana. Para te mostrar um pouco da nossa incrível fauna, neste post do PeritoAnimal nos focamos em listar aqueles animais brasileiros: nativos e endêmicos que são icônicos na nossa história, além dos animais que só existem no Brasil. Continue lendo e encante-se com a magnitude de cada um deles! Também lhe pode interessar: 15 animais ameaçados de extinção no Brasil Índice Fauna brasileira Animais brasileiros Animais que só existem no Brasil 8 cachorros brasileiros Outros animais da fauna brasileira Fauna brasileira De acordo com o Instituto Chico Mendes, [1] o Brasil administra o maior patrimônio de biodiversidade do mundo. Em números isso se traduz em 120 mil espécies de invertebrados e 8930 espécies de vertebrados, aproximadamente, dentre eles são: 734 espécies de mamíferos;"
25257,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-53-03652_0,85.536163,1,7347298,"Fauna do Brasil – Wikipédia, a enciclopédia livre [ocultar] 1 língua Fauna do Brasil Origem: Wikipédia, a enciclopédia livre. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico) A Fauna do Brasil envolve o conjunto de espécies de animais distribuídas por todo o território brasileiro. Na selva amazônica existe uma abundante fauna de peixes e mamíferos aquáticos que habitam os rios e lagos. As espécies mais conhecidas são o pirarucu e o peixe-boi (este em via de extinção). Nas várzeas existem jacarés e tartarugas (também ameaçados de extinção), bem como algumas espécies de anfíbios, notadamente a lontra e a capivara e certas serpentes, como a sucuri. Nas florestas em geral predominam a anta, a onça, os macacos, o preguiça, o caititu, a jiboia, a sucuri, os papagaios, araras e tucanos e uma imensa variedade de insetos e aracnídeos ."
45087,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_2,85.465218,1,5877601,"Na Mata Atlântica, nos deparamos com mais de 1020 espécies apenas entre as aves, das quais 

In [76]:
bm25_e5_run_df['Q0'] = "Q0"

In [77]:
bm25_e5_run_df['comment'] = "gpt-4-1106-preview"

In [115]:
bm25_e5_run_df['index'] = np.tile(list(range(1, 11)), 50)

In [116]:
bm25_e5_run_df.head(12)

,query,passage_id,score,query_id,colbertx_id,passage,Q0,comment,index
45085,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_0,88.380936,1,5877599,"Fauna do Brasil - Toda Matéria Biologia › Zoologia Fauna do Brasil A fauna do Brasil é provavelmente a que possui a maior diversidade em espécies de animais vertebrados, anfíbios e primatas do globo terrestre. Existem três vezes mais espécies de peixes de água doce no Brasil do que em qualquer lugar no mundo. As aves também são muito variadas, colocando o Brasil como o país com a maior variedade de espécies na América do Sul e o terceiro do mundo em espécies de mamíferos. De igual modo, a Floresta Atlântica está entre as cinco regiões do planeta com mais espécies nativas, das quais 17% são aves e 10% são anfíbios e mamíferos encontrados somente na fauna brasileira. Vale citar que os animais da fauna brasileira são, via de regra, de pequeno porte. A importância científica da fauna brasileira é indescritível, pois contém a maior biodiversidade do mundo (cerca de 100 mil espécies de invertebrados, 8200 espécies vertebrados (713 mamíferos, 1826 aves, 721 répteis, 875 anfíbios, 2800 peixes continentais e 1300 peixes marinhos) das quais 627 já estão na lista negra das espécies ameaçadas de extinção (394 espécies terrestres e 233 espécies aquáticas).",Q0,gpt-4-1106-preview,1
45086,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-96-00412_0,85.875961,1,4583617,"15 ANIMAIS BRASILEIROS: Características e curiosidades Animais brasileiros: nativos, endêmicos e em risco de extinção Fauna quer dizer o conjunto de espécies que vivem em uma determinada região. Por isso, é importante saber diferenciar que quando falamos sobre a fauna brasileira, estamos falando sobre todas as espécies que habitam o Brasil, mas não necessariamente apenas de espécies endêmicas ou nativas, já que alguns animais são considerados espécies invasoras e/ou foram introduzidos pela pessoa humana. Para te mostrar um pouco da nossa incrível fauna, neste post do PeritoAnimal nos focamos em listar aqueles animais brasileiros: nativos e endêmicos que são icônicos na nossa história, além dos animais que só existem no Brasil. Continue lendo e encante-se com a magnitude de cada um deles! Também lhe pode interessar: 15 animais ameaçados de extinção no Brasil Índice Fauna brasileira Animais brasileiros Animais que só existem no Brasil 8 cachorros brasileiros Outros animais da fauna brasileira Fauna brasileira De acordo com o Instituto Chico Mendes, [1] o Brasil administra o maior patrimônio de biodiversidade do mundo. Em números isso se traduz em 120 mil espécies de invertebrados e 8930 espécies de vertebrados, aproximadamente, dentre eles são: 734 espécies de mamíferos;",Q0,gpt-4-1106-preview,2
25257,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-53-03652_0,85.536163,1,7347298,"Fauna do Brasil – Wikipédia, a enciclopédia livre [ocultar] 1 língua Fauna do Brasil Origem: Wikipédia, a enciclopédia livre. Esta página cita fontes, mas estas não cobrem todo o conteúdo. Ajude a inserir referências. Conteúdo não verificável poderá ser removido .— Encontre fontes: Google (notícias, livros e acadêmico) A Fauna do Brasil envolve o conjunto de espécies de animais distribuídas por todo o território brasileiro. Na selva amazônica existe uma abundante fauna de peixes e mamíferos aquáticos que habitam os rios e lagos. As espécies mais conhecidas são o pirarucu e o peixe-boi (este em via de extinção). Nas várzeas existem jacarés e tartarugas (também ameaçados de extinção), bem como algumas espécies de anfíbios, notadamente a lontra e a capivara e certas serpentes, como a sucuri. Nas florestas em geral predominam a anta, a onça, os macacos, o preguiça, o caititu, a jiboia, a sucuri, os papagaios, araras e tucanos e uma imensa variedade de insetos e aracnídeos .",Q0,gpt-4-1106-preview,3
45087,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_2,85.465218,1,5877601,"Na

In [81]:
bm25_e5_run_df[['query_id', 'Q0', "passage_id", "index", "score", "comment"]].to_csv("tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt", header=None, index=False, sep=" ")

#### Compute NDCG@10

In [221]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    tests/evaluation_04_retrievers_00_009_sampled_50_queries_gpt-4-1106-preview_20231211_qrel.txt tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt

In [222]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

In [102]:
!/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    tests/evaluation_04_retrievers_00_009_sampled_50_queries_gpt-4-1106-preview_20231211_shuffled_qrel.txt tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt

ndcg_cut_10           	all	0.6820


### Prepare BM25 + mT5 results in run format

In [40]:
mt5_df = pd.read_csv(BM25_MT5, sep="\t")

In [41]:
mt5_df

,query,doc_id,passage-by-bm25-mt5,query_id
0,Qual objetivo da Enel Brasil?,clueweb22-pt0000-58-10513_2,"Sendo assim, em linha com o modelo de Criação de Valor Compartilhado (Creating Shared Value - CSV) adotado pela companhia mundialmente, temos o objetivo de associar o desenvolvimento do nosso negócio às necessidades das comunidades brasileiras e realizamos diversas iniciativas para possibilitar a geração de renda, como cursos, oficinas e outros projetos sociais. Além disso, nos comprometemos explicitamente a alcançar quatro dos 17 Objetivos de Desenvolvimento Sustentável (ODS) da ONU: amplo acesso à energia limpa e acessível, apoio à educação de qualidade, contribuição para o desenvolvimento socioeconômico e combate contra as alterações climáticas. Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100
1,Qual objetivo da Enel Brasil?,clueweb22-pt0000-81-12555_1,"A Enel Brasil foi constituída em 2005 com o objeto de centralizar as participações societárias do grupo Enel no país e criar, por meio de uma holding de empresas, uma plataforma sólida de gestão dos ativos que preparasse a companhia para futuras expansões. Em 2007, a Enel SpA, controladora indireta da Enel Brasil, iniciou a consolidação de sua participação sobre a holding brasileira. Por meio de uma oferta pública no exterior, a Enel SpA adquiriu a totalidade das ações de emissão da espanhola Endesa – na época acionista controladora indireta da Enel Brasil. Em 2009, o controle da Enel SpA sobre a Enel Brasil foi ampliado, com a aquisição de 92,06% do capital social e votante da Endesa. Finalmente, em 2014, a Enel SpA adquiriu – por meio de uma operação de compra e venda de ações – a participação da Endesa em sociedades na América Latina, o que incluiu a Enel Brasil. Desde então, a empresa espanhola foi excluída da estrutura acionária no país. Apesar da holding Enel Brasil ter nascido em 2005, a trajetória do grupo Enel no país começou quase uma década antes, em 1996, com a aquisição da distribuidora de energia elétrica CERJ em leilão de privatização. Novas aquisições foram realizadas ao longo dos anos, garantindo a ampliação e diversificação do portfólio.",100
2,Qual objetivo da Enel Brasil?,clueweb22-pt0000-39-18411_1,"Inovação e Sustentabilidade inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100
3,Qual objetivo da Enel Brasil?,clueweb22-pt0002-00-04601_1,"inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique 

In [134]:
mt5_df.groupby('query_id').count()

,query,doc_id,passage-by-bm25-mt5
query_id,,,
0,1000,1000,1000
1,1000,1000,1000
2,1000,1000,1000
3,1000,1000,1000
4,1000,1000,1000
5,1000,1000,1000
6,1000,1000,1000
7,1000,1000,1000
8,1000,1000,1000


In [133]:
mt5_df.drop_duplicates('query_id')[['query_id', 'query']]

,query_id,query
0,100,Qual objetivo da Enel Brasil?
1000,101,Qual a abordagem de mineração da moeda digital Chia (XCH)?
2000,102,Quais produtos são revendidos pela empresa Casa da Coifa?
3000,103,Em que ano surgiu a Universities and Colleges Admissions Service (UCAS)?
4000,104,Quais etapas para a inscrição pelo Colleges Admissions Service (UCAS)?
5000,105,Onde está localizada a Praça XV de Novembro?
6000,106,Quais os monumentos históricos situados na Praça XV de Novembro?
7000,107,Quando foi inaugurada a Estátua Equestre de Dom João VI?
8000,108,Mercado de trabalho para profissionais formados em Engenharia Química
9000,109,O que é o programa International Office da universidade São Judas?


In [135]:
mt5_df['Q0'] = "Q0"
mt5_df['comment'] = "BM25+mT5"

In [145]:
mt5_top_10_df = mt5_df.groupby('query_id').head(10).reset_index(drop=True)

In [146]:
mt5_top_10_df.shape

(2000, 6)

In [147]:
mt5_top_10_df['position'] = np.tile(list(range(1, 11)), 200)

In [149]:
mt5_top_10_df['score'] = np.tile(list(range(100, 0, -10)), 200)

In [150]:
mt5_top_10_df.head(12)

,query,doc_id,passage-by-bm25-mt5,query_id,Q0,comment,position,score
0,Qual objetivo da Enel Brasil?,clueweb22-pt0000-58-10513_2,"Sendo assim, em linha com o modelo de Criação de Valor Compartilhado (Creating Shared Value - CSV) adotado pela companhia mundialmente, temos o objetivo de associar o desenvolvimento do nosso negócio às necessidades das comunidades brasileiras e realizamos diversas iniciativas para possibilitar a geração de renda, como cursos, oficinas e outros projetos sociais. Além disso, nos comprometemos explicitamente a alcançar quatro dos 17 Objetivos de Desenvolvimento Sustentável (ODS) da ONU: amplo acesso à energia limpa e acessível, apoio à educação de qualidade, contribuição para o desenvolvimento socioeconômico e combate contra as alterações climáticas. Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100,Q0,BM25+mT5,1,100
1,Qual objetivo da Enel Brasil?,clueweb22-pt0000-81-12555_1,"A Enel Brasil foi constituída em 2005 com o objeto de centralizar as participações societárias do grupo Enel no país e criar, por meio de uma holding de empresas, uma plataforma sólida de gestão dos ativos que preparasse a companhia para futuras expansões. Em 2007, a Enel SpA, controladora indireta da Enel Brasil, iniciou a consolidação de sua participação sobre a holding brasileira. Por meio de uma oferta pública no exterior, a Enel SpA adquiriu a totalidade das ações de emissão da espanhola Endesa – na época acionista controladora indireta da Enel Brasil. Em 2009, o controle da Enel SpA sobre a Enel Brasil foi ampliado, com a aquisição de 92,06% do capital social e votante da Endesa. Finalmente, em 2014, a Enel SpA adquiriu – por meio de uma operação de compra e venda de ações – a participação da Endesa em sociedades na América Latina, o que incluiu a Enel Brasil. Desde então, a empresa espanhola foi excluída da estrutura acionária no país. Apesar da holding Enel Brasil ter nascido em 2005, a trajetória do grupo Enel no país começou quase uma década antes, em 1996, com a aquisição da distribuidora de energia elétrica CERJ em leilão de privatização. Novas aquisições foram realizadas ao longo dos anos, garantindo a ampliação e diversificação do portfólio.",100,Q0,BM25+mT5,2,90
2,Qual objetivo da Enel Brasil?,clueweb22-pt0000-39-18411_1,"Inovação e Sustentabilidade inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO Não é cliente Enel? Clique aqui e conheça o nosso Grupo",100,Q0,BM25+mT5,3,80
3,Qual objetivo da Enel Brasil?,clueweb22-pt0002-00-04601_1,"inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione 

In [151]:
mt5_top_10_df[['query_id', 'Q0', "doc_id", "position", "score", "comment"]].to_csv("tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt", header=None, index=False, sep=" ")

#### Compute NDCG@10

In [223]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    tests/evaluation_04_retrievers_00_009_sampled_50_queries_gpt-4-1106-preview_20231211_qrel.txt tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt

In [224]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare SPLADE results in run format

In [140]:
splade_df = pd.read_csv(SPLADE, sep="\t")

In [141]:
splade_df

,query_id,Q0,colbertx_id,position,score,comment,clueweb_id
0,0,Q0,6124160,1,18.215471,SPLADE-PT,clueweb22-pt0001-27-16321_1
1,0,Q0,9394964,2,18.258245,SPLADE-PT,clueweb22-pt0001-97-01569_0
2,1,Q0,9394964,706,18.854155,SPLADE-PT,clueweb22-pt0001-97-01569_0
3,68,Q0,9394964,921,18.207718,SPLADE-PT,clueweb22-pt0001-97-01569_0
4,0,Q0,319340,3,17.736769,SPLADE-PT,clueweb22-pt0000-06-04821_0
...,...,...,...,...,...,...,...
199995,199,Q0,6138175,995,12.434690,SPLADE-PT,clueweb22-pt0001-28-00764_1
199996,199,Q0,3132292,996,12.457484,SPLADE-PT,clueweb22-pt0000-65-16234_4
199997,199,Q0,1040828,997,12.447216,SPLADE-PT,clueweb22-pt0000-21-11900_3
199998,199,Q0,5391434,999,12.460681,SPLADE-PT,clueweb22-pt0001-12-09424_3


In [142]:
splade_sorted_df = splade_df.sort_values(["query_id", "score"], ascending=[True, False]).groupby('query_id').head(10)

In [143]:
splade_sorted_df

,query_id,Q0,colbertx_id,position,score,comment,clueweb_id
802,0,Q0,4178525,360,22.945511,SPLADE-PT,clueweb22-pt0000-87-14633_2
625,0,Q0,7135402,280,22.692141,SPLADE-PT,clueweb22-pt0001-48-11769_116
11,0,Q0,5539594,6,22.510916,SPLADE-PT,clueweb22-pt0001-15-15106_2
278,0,Q0,6946573,122,22.492905,SPLADE-PT,clueweb22-pt0001-44-12879_4
841,0,Q0,5977156,382,22.464260,SPLADE-PT,clueweb22-pt0001-24-16393_2
...,...,...,...,...,...,...,...
198667,199,Q0,1827001,70,23.841969,SPLADE-PT,clueweb22-pt0000-38-01029_7
198569,199,Q0,2396198,641,23.648439,SPLADE-PT,clueweb22-pt0000-50-08513_1
198507,199,Q0,4244278,687,23.571337,SPLADE-PT,clueweb22-pt0000-88-19545_11
198511,199,Q0,8308524,11,23.179789,SPLADE-PT,clueweb22-pt0001-73-07268_17


In [144]:
splade_sorted_df['position'] = np.tile(list(range(1, 11)), 200)

In [118]:
splade_sorted_df.head(12)

,query_id,Q0,colbertx_id,position,score,comment,clueweb_id
802,0,Q0,4178525,1,22.945511,SPLADE-PT,clueweb22-pt0000-87-14633_2
625,0,Q0,7135402,2,22.692141,SPLADE-PT,clueweb22-pt0001-48-11769_116
11,0,Q0,5539594,3,22.510916,SPLADE-PT,clueweb22-pt0001-15-15106_2
278,0,Q0,6946573,4,22.492905,SPLADE-PT,clueweb22-pt0001-44-12879_4
841,0,Q0,5977156,5,22.464260,SPLADE-PT,clueweb22-pt0001-24-16393_2
1462,0,Q0,8000654,6,22.444901,SPLADE-PT,clueweb22-pt0001-67-00120_5
588,0,Q0,9609753,7,22.388660,SPLADE-PT,clueweb22-pt0002-01-16686_0
766,0,Q0,473093,8,22.112490,SPLADE-PT,clueweb22-pt0000-09-13182_0
947,0,Q0,4077560,9,22.045551,SPLADE-PT,clueweb22-pt0000-85-17244_5
489,0,Q0,4138910,10,21.752874,SPLADE-PT,clueweb22-pt0000-86-19982_33


In [119]:
splade_sorted_df[['query_id', 'Q0', "clueweb_id", "position", "score", "comment"]].to_csv("tests/splade_pt_result_with_clueweb_id_run.txt", header=None, index=False, sep=" ")

#### Compute NDCG@10

In [225]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    tests/evaluation_04_retrievers_00_009_sampled_50_queries_gpt-4-1106-preview_20231211_qrel.txt tests/splade_pt_result_with_clueweb_id_run.txt

In [226]:
retrievers_results.append({'retriever':'SPLADE_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare ColbertX results in run format

In [108]:
colbertx_df = pd.read_csv(COLBERTX, sep="\t")

In [109]:
colbertx_df

,query_id,colbertx_id,doc_position,doc_score,score_znorm,part,clueweb_id
0,0,4138910,1,28.797396,3.723518,2,clueweb22-pt0000-86-19982_33
1,1,4138910,1,28.135212,3.365722,2,clueweb22-pt0000-86-19982_33
2,25,4138910,671,21.382965,-0.282701,2,clueweb22-pt0000-86-19982_33
3,26,4138910,476,20.872330,-0.558611,2,clueweb22-pt0000-86-19982_33
4,54,4138910,31,26.231224,2.336945,2,clueweb22-pt0000-86-19982_33
...,...,...,...,...,...,...,...
3193552,199,7325692,722,19.289238,-1.382245,3,clueweb22-pt0001-52-15266_355
3193553,199,6825557,723,19.273064,-1.391039,3,clueweb22-pt0001-42-02150_2
3193554,199,6358548,654,19.174335,-1.444718,3,clueweb22-pt0001-32-16864_1
3193555,199,2829893,648,19.097916,-1.486261,1,clueweb22-pt0000-59-19225_192


In [124]:
colbertx_sorted_df = colbertx_df.sort_values(["query_id", "score_znorm"], ascending=[True, False]).groupby('query_id').head(10)

In [125]:
colbertx_sorted_df.shape

(2000, 7)

In [126]:
colbertx_sorted_df['doc_position'] = np.tile(list(range(1, 11)), 200)

In [128]:
colbertx_sorted_df['Q0'] = 'Q0'

In [129]:
colbertx_sorted_df['comment'] = 'colbertx'

In [127]:
colbertx_sorted_df.head(12)

,query_id,colbertx_id,doc_position,doc_score,score_znorm,part,clueweb_id
0,0,4138910,1,28.797396,3.723518,2,clueweb22-pt0000-86-19982_33
7,0,8157162,2,28.422802,3.516794,4,clueweb22-pt0001-70-06195_8
17,0,7828446,3,28.384157,3.495905,3,clueweb22-pt0001-63-03254_3
23,0,3392707,4,28.252335,3.415572,1,clueweb22-pt0000-71-04006_2
31,0,2325385,5,28.157360,3.364606,1,clueweb22-pt0000-48-18850_1
39,0,8430956,6,28.131063,3.359750,4,clueweb22-pt0001-76-07787_4
44,0,6865798,7,28.121971,3.355167,3,clueweb22-pt0001-42-18761_18
52,0,3577057,8,28.106983,3.337572,1,clueweb22-pt0000-75-07965_7
70,0,3917442,9,28.045986,3.304839,1,clueweb22-pt0000-82-13187_0
81,0,7987527,10,28.024965,3.303096,3,clueweb22-pt0001-66-11652_2


In [130]:
colbertx_sorted_df[['query_id', 'Q0', "clueweb_id", "doc_position", "score_znorm", "comment"]].to_csv("tests/colbertx_all_znormalized_validation_scores_000_199_run.txt", header=None, index=False, sep=" ")

#### Compute NDCG@10

In [227]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    tests/evaluation_04_retrievers_00_009_sampled_50_queries_gpt-4-1106-preview_20231211_qrel.txt tests/colbertx_all_znormalized_validation_scores_000_199_run.txt

In [228]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [229]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.2967
1,BM25+E5,0.6820
2,BM25+mT5,0.7597
3,SPLADE_pt-BR,0.6119
4,ColBERT-X_mMARCO_pt-BR,0.6649


### Check execution log against BM25 retrievals

In [230]:
import pickle

In [231]:
with open("tests/execution_rev_2.1_sampled_50_queries_gpt-4-1106-preview_20231211.pkl", "rb") as input_file:
    logs = pickle.load(input_file)

In [233]:
already_analized = 0

for i, row in bm25_top_10_df.iterrows():
    if (row['query'] + "_" + row['passagem-id']) in logs['evaluation_cache']:
        print("Already analized...")

        already_analized += 1

Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


In [234]:
already_analized 

181

In [235]:
already_analized / bm25_top_10_df.shape[0]

0.09095477386934674

#### Filter the sampled 50 queries for GPT-4 evaluation

In [242]:
bm25_eval_df = pd.read_csv("tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_for_evaluation.tsv", sep='\t')

In [243]:
bm25_eval_df

,query,passage_id,passage,query_id,score
0,Qual objetivo da Enel Brasil?,clueweb22-pt0001-44-18816_1,"Nos últimos anos, o atendimento ao cliente Enel, tem se aprimorado, com o objetivo de facilitar ainda mais a vida dos consumidores. Com isso, grande parte dos serviços de atendimento ao cliente Enel, podem ser realizados de maneira prática, por meio da internet. Através do aplicativo Enel- Que está disponível para download em aparelhos com Android ou IO Nas Redes Sociais – Como na página do Facebook (Enel clientes Brasil). Ou no Twitter (@ Enel clientes BR). Por meio do site- Na página da Enel, (www.enel.com.br/pt.html). O atendimento Enel ao cliente, pode ser feito por meio de um assistente virtual. Veja o vídeo: Outras formas de atendimento Além de poder receber o atendimento por meio dos canais online, você também pode obter atendimento nos Estados onde a Enel Brasil está presente. Por meio do telefone de atendimento Enel 0800, ou indo pessoalmente a uma loja de atendimento mais próxima da sua casa. Número 0800 para atendimento da Enel Pelo telefone de atendimento ao cliente da Enel, é possível esclarecer suas dúvidas e ainda relatar algum problema, entre outros serviços. Veja quais são os telefones da Enel atendimento ao cliente, aproveite para anotar o número da Enel em seu estado: Atendimento ao cliente Enel Ceará: 0800 285 0196",100,100
1,Qual objetivo da Enel Brasil?,clueweb22-pt0001-44-10630_6,"A ouvidoria é um canal de segunda instância que tem o objetivo de ouvir e tentar resolver problemas/solicitações que os canais convencionais de atendimento não foram capazes de solucionar. Caso nem mesmo a ouvidoria da Enel seja capaz de resolver em até 15 dias sua manifestação, é indicado que você busque ajuda dos órgãos responsáveis pela defesa do consumidor. 8. Redes sociais da Enel Brasil Outra forma de buscar falar com Enel é através de algumas redes sociais. A empresa geralmente auxilia os clientes pelo Facebook e Twitter, seja em comentários públicos ou mensagens privadas. Além disso, elas são boas alternativas para que você acompanhe as principais novidades e mudanças da distribuidora de energia. Os links da redes sociais da Enel Brasil são: Facebook Twitter Instagram (não presta suporte) 9. Resumo dos contatos da Enel Veja na tabela a seguir um resumo com alguns dos principais contatos oficiais da Enel abordados no artigo. Além disso, também listamos contatos que a empresa não disponibiliza para que você não caia em golpes. Se ficar dúvidas sobre algum dos meios de atendimento abaixo, leia os tópicos do artigo novamente e verá informações mais detalhadas. Fale com a Enel Contato SÃO PAULO Telefone Enel São Paulo (SAC) 0800-727-2120 Telefone para ligações de emergência",100,90
2,Qual objetivo da Enel Brasil?,clueweb22-pt0001-05-18926_3,"O medidor inteligente, aprovado pelo Instituto Nacional de Metrologia, Qualidade e Tecnologia (INMETRO), foi desenvolvido para atender às necessidades locais, respeitando as características da rede elétrica e os requisitos das normas técnicas. Globalmente, a Enel já instalou mais de 45 milhões de medidores inteligentes em países como Argentina, Chile, Colômbia, Itália, Peru, Romênia e Espanha. Sobre a Enel Distribuição São Paulo A Enel Distribuição São Paulo é uma empresa da multinacional de energia Enel. A companhia é a segunda maior distribuidora do país, respondendo por 10,3% de toda energia distribuída no Brasil e atendendo 7,4 milhões de unidades consumidoras em 24 municípios da região metropolitana de São Paulo, incluindo a capital paulista, principal centro econômico-financeiro do Brasil. A estratégia de atuação da Enel é baseada no seu Plano de Sustentabilidade e nos compromissos assumidos com os Objetivos de Desenvolvimento Sustentável da Agenda 2030 da ONU. |8 de Setembro de 2021 Enel São Paulo instalou 89 mil medidores inteligentes em bairros da Zona Oeste Pdf (0.1MB) DOWNLOAD [EDIT TITLE]",100,80
3,Qual objetivo da Enel Brasil?,clueweb22-pt0000-62-20856_1,"Serviço: Loja de At

In [244]:
sampled_queries_df = pd.read_csv("../trabalho_final/final_validation_queries/50-dataset-doccano-queries-passagens-by-BM25-mT5_with_query_id.tsv", sep="\t")

In [245]:
sampled_queries_df.shape

(500, 5)

In [246]:
sampled_queries_list_df = sampled_queries_df.drop_duplicates('query')[['query_id', 'query']].reset_index(drop=True)

In [247]:
bm25_eval_df = bm25_eval_df.merge(sampled_queries_list_df, left_on="query", right_on="query")

In [249]:
bm25_eval_df.shape

(500, 6)

In [251]:
already_analized = 0

for i, row in bm25_eval_df.iterrows():
    if (row['query'] + "_" + row['passage_id']) in logs['evaluation_cache']:
        print("Already analized...")

        already_analized += 1

Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


In [252]:
already_analized

181

In [253]:
already_analized / bm25_eval_df.shape[0]

0.362

In [254]:
bm25_eval_df.to_csv("tests/bm25_docs_000_009_50_sampled_queries.tsv", sep='\t', index=False)